<a href="https://colab.research.google.com/github/architb1703/Toxic_Span/blob/Abhay/roberta_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle

In [ ]:
path = 'drive/MyDrive/Toxic Spam/augmented.pkl'

In [ ]:
with open(path,'rb') as f:
  aug_data=pickle.load(f)


In [ ]:
print(aug_data)

                                            token_final                                       target_final
0     [Only, a, fool, would, not, obviously, preside...      [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1     [cops, are, dumbasses, any, male, from, is, a,...  [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
2     [Well, stated, 'Donald, dumb-azz, be, Vladimir...  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     [So, ,, the, is, on, white, supremacists, whil...  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     [is, ,, another, sycophant, and, lick, The, on...  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
...                                                 ...                                                ...
4559  [Yeah, ,, your, idiot, let, 's, be, stupid, ra...  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
4560  [do, You, Beav, wonder, n't, ;, you, make, a, ...  [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...
4561  [As, for, Clinton, order, it, '

In [ ]:
train_path = 'drive/MyDrive/Ensemble/train.pkl'
val_path = 'drive/MyDrive/Ensemble/val.pkl'

with open(train_path, 'rb') as f:
  train_data = pickle.load(f)
  f.close()

with open(val_path, 'rb') as f:
  val_data = pickle.load(f)
  f.close()

In [ ]:
import pandas as pd

In [ ]:
final_train_data = pd.concat([train_data,aug_data],ignore_index=True)
print(len(final_train_data))

10915


In [ ]:
X_train = final_train_data['token_final']
X_val = val_data['token_final']
Y_train = final_train_data['target_final']
Y_val = val_data['target_final']

In [ ]:
print(len(X_train),len(Y_train))

10915 10915


In [ ]:
import torch
if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers==4.0.0
!pip install seqeval
!pip install urllib3 --upgrade

     |████████████████████████████████| 1.4MB 22.0MB/s 
     |████████████████████████████████| 2.9MB 55.2MB/s 
     |████████████████████████████████| 890kB 56.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=866b33898ba8bc4058113b467aba4c3e724422a1b666747fc5adb87459c2b017
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 51kB 6.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=58418ca51099c0126a3ac2b1684d7761b5026a0de933e5482109fdcf14aef2da
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
     |████████████████████████████████| 143kB 21.6MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERR

In [ ]:
#Importing librariesCLASSES = {'0':0, '1':1, '[PAD]':2}

import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm, trange

import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchtext import data
import transformers
from transformers import RobertaTokenizer,  RobertaForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup

from keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix

np.random.seed(42)
torch.manual_seed(12)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
def tokenize_roberta(x, y):
  sentence = []
  labels = [0]
  for word, label in zip(x, y):
    tokenized_word = tokenizer.tokenize(word)
    sentence.extend(tokenized_word)
    labels.extend([label for i in range(len(tokenized_word))])
  labels.append(0)
  return(sentence, labels)

In [ ]:
print(tokenize_roberta(X_train[0],Y_train[0]))

(['Only', 'a', 'f', 'ool', 'would', 'not', 'be', 'against', 'this', 'president', 'He', 'is', 'mass', 'ively', 'and', 'ob', 'viously', 'un', 'fit', 'for', 'office'], [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
  len_train = len(X_train)
  len_val = len(X_val)

  for i in range(len_train):
    X_train[i], Y_train[i] = tokenize_roberta(X_train[i], Y_train[i])

  for i in range(len_val):
    X_val[i], Y_val[i] = tokenize_roberta(X_val[i], Y_val[i])

In [ ]:
#Calculating class weights for training on weighted cross-entropy
ones = 0
zeros = 0
total = 0
for y in Y_train:
  ones += np.sum(np.array(y))
  zeros += len(y) - np.sum(np.array(y))
  total += len(y)
for y in Y_val:
  ones += np.sum(np.array(y))
  zeros += len(y) - np.sum(np.array(y))
  total += len(y)
class_weights = torch.tensor([zeros/zeros, zeros/ones], dtype=torch.float32)

In [ ]:
#We use the class 2 to pad the labels tensors and it also allows us to find the mask easily
CLASSES = {'0':0, '1':1, '[PAD]':2}
MAX_LEN = 500
BATCH_SIZE = 4
#Convert tokens to token_ids for bert_model, add special tokens to the sequences, pad the sequences
X_train_id = pad_sequences([tokenizer.encode(text) for text in X_train], maxlen = MAX_LEN, dtype='long', value=0.0, truncating='post', padding = 'post')
Y_train_id = pad_sequences(Y_train, maxlen=MAX_LEN, value=CLASSES['[PAD]'], dtype='long', truncating='post', padding='post')
X_val_id = pad_sequences([tokenizer.encode(text) for text in X_val], maxlen = MAX_LEN, dtype='long', value=0.0, truncating='post', padding = 'post')
Y_val_id = pad_sequences(Y_val, maxlen=MAX_LEN, value=CLASSES['[PAD]'], dtype='long', truncating='post', padding='post')

In [ ]:
def get_attention_mask(x):
  return([[(i!=0) for i in text] for text in x])

In [ ]:
#Generate mask for training so that padding tokens are not considered while training
attention_mask_train = get_attention_mask(X_train_id)
attention_mask_val = get_attention_mask(X_val_id)

In [ ]:
X_train_id = torch.tensor(X_train_id)
Y_train_id = torch.tensor(Y_train_id)
X_val_id = torch.tensor(X_val_id)
Y_val_id = torch.tensor(Y_val_id)
attention_mask_train = torch.tensor(attention_mask_train)
attention_mask_val = torch.tensor(attention_mask_val)

In [ ]:
#Initialize dataloaders for train and validation data which will give batches of data, labels, masks
train_data = TensorDataset(X_train_id, attention_mask_train, Y_train_id)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(X_val_id, attention_mask_val, Y_val_id)
val_sampler = RandomSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import RobertaForTokenClassification
model = RobertaForTokenClassification.from_pretrained('roberta-base', num_labels=2)
model.cuda()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [ ]:
#Initialize AdamW optimizer (add weight decay for regularization) 
FINE_TUNING = True
if FINE_TUNING:
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
                                  {'params' : [p for n,p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate' : 0.01},
                                  {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],'weight_decay_rate': 0.0}]
else:
  param_optimizer = list(model.classifier.named_parameters())
  optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, eps=1e-8)

In [ ]:
#Initialize scheduler (helps to gradually decrease the learning rate)
epochs = 3
max_grad_norm = 1.0

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = criterion.to(device)

In [ ]:
#Training code

train_loss, val_loss = [], []
train_acc, val_acc = [], []
train_f1, val_f1 = [], []

l = 100

for epoch in range(epochs):
  print(f"Epoch {epoch}")
  model.train()
  t_loss, t_acc = 0, 0
  predictions, true_labels = [], []
  for step, batch in enumerate(tqdm(train_dataloader, desc='Train')):
    batch = tuple(t.to(device) for t in batch)
    b_input_id, b_input_mask, b_labels = batch
    model.zero_grad()
    outputs = model(b_input_id, token_type_ids=None, attention_mask=b_input_mask, labels = b_labels)
    # loss = outputs[0]

    # Code for using weighted cross-entropy loss
    active_loss = b_input_mask.view(-1) == 1
    active_logits = outputs[1].view(-1, 2)
    active_labels = torch.where(active_loss, b_labels.view(-1), torch.tensor(criterion.ignore_index).type_as(b_labels))
    loss = criterion(active_logits, active_labels)

    loss.backward()
    t_loss += loss.item()
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    optimizer.step()
    scheduler.step()

    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)
    
  print(f"Train Loss : {t_loss/len(train_dataloader)}")
  train_loss.append(t_loss/len(train_dataloader))
  pred_tags = [[p_i for p_i, l_i in zip(p,l) if l_i!=2] for p,l in zip(predictions, true_labels)]
  pred_tags = [p_i for p in pred_tags for p_i in p[1:-1]]
  # pred_tags = [p_i for p, l in zip(predictions, true_labels)
  #                               for p_i, l_i in zip(p, l) if l_i != 2]
  # valid_tags = [l_i for l in true_labels
  #                               for l_i in l if l_i != 2]
  valid_tags = [[l_i for l_i in l if l_i!=2] for l in true_labels]
  valid_tags = [l_i for l in valid_tags for l_i in l[1:-1]]
  train_acc.append(accuracy_score(pred_tags, valid_tags))
  train_f1.append(f1_score(pred_tags, valid_tags))
  print("Train Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Train F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
  print()

  model.eval()
  v_loss, v_accuracy = 0, 0
  predictions , true_labels = [], []
  for batch in tqdm(val_dataloader, desc="Val"):
    batch = tuple(t.to(device) for t in batch)
    b_input_id, b_input_mask, b_labels = batch
    with torch.no_grad():
      outputs = model(b_input_id, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    
    # loss = outputs[0]

    active_loss = b_input_mask.view(-1) == 1
    active_logits = outputs[1].view(-1, 2)
    active_labels = torch.where(active_loss, b_labels.view(-1), torch.tensor(criterion.ignore_index).type_as(b_labels))
    loss = criterion(active_logits, active_labels)
    
    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    v_loss += loss.item()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)
  
  v_loss = v_loss/len(val_dataloader)
  val_loss.append(v_loss)
  if(v_loss < l):
    l = v_loss
    print("Model Checkpoint")
  torch.save(model, f'drive/MyDrive/Toxic Spam/1612augroberta{epoch}.pt')
  
  print(f"Validation Loss : {v_loss}")
  pred_tags = [[p_i for p_i,l_i in zip(p,l) if l_i!=2] for p,l in zip(predictions, true_labels)]
  pred_tags = [p_i for p in pred_tags for p_i in p[1:-1]]
  # pred_tags = [p_i for p, l in zip(predictions, true_labels)
  #                               for p_i, l_i in zip(p, l) if l_i != 2]
  # valid_tags = [l_i for l in true_labels
  #                               for l_i in l if l_i != 2]
  valid_tags = [[l_i for l_i in l if l_i!=2] for l in true_labels]
  valid_tags = [l_i for l in valid_tags for l_i in l[1:-1]]
  print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
  val_acc.append(accuracy_score(pred_tags, valid_tags))
  val_f1.append(f1_score(pred_tags, valid_tags))
  print()

Train:   0%|          | 0/2729 [00:00<?, ?it/s]

Epoch 0


Train: 100%|██████████| 2729/2729 [20:04<00:00,  2.27it/s]


Train Loss : 0.45345360395447726
Train Accuracy: 0.8821493742684636


Val:   1%|          | 1/199 [00:00<00:35,  5.62it/s]

...

Validation Accuracy: 0.8933350260251365
Validation F1-Score: 0.5305620739747459

Epoch 2


Train: 100%|██████████| 2729/2729 [20:23<00:00,  2.23it/s]


Train Loss : 0.3537612294879926
Train Accuracy: 0.9200764209520776
Train F1-Score: 0.6137939197186433



Val: 100%|██████████| 199/199 [00:30<00:00,  6.49it/s]


Validation Loss : 0.5259276471433029
Validation Accuracy: 0.8928526088612416
Validation F1-Score: 0.5284916201117319



In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1911 MB |    4929 MB |   92122 GB |   92120 GB |\n|       from large pool |    1909 MB |    4927 MB |   92089 GB |   92087 GB |\n|       from small pool |       1 MB |       3 MB |      32 GB |      32 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    1911 MB |    4929 MB |   92122 GB |   92120 GB |\n|       from large pool |    1909 MB |    4927 MB |

In [ ]:
#Class to take in tokenizer and model along with input data and calculate the span average f1 score
class SpanAvgF1():
  def __init__(self, model, tokenizer, X, y, spans, target_spans, maxlen=500):
    self.model = model
    self.model.cuda()
    self.tokenizer = tokenizer
    self.maxlen = maxlen
    self.X = X
    self.y = y
    self.spans = spans
    self.target_spans = target_spans
    self.prepare_data()

    self.X = torch.tensor(self.X)
    self.y = torch.tensor(self.y)
    self.attention_mask = torch.tensor(self.attention_mask)

    data = TensorDataset(self.X, self.attention_mask, self.y)
    self.dataloader = DataLoader(data, batch_size=16, shuffle=False)
  
  def tokenize_data(self, x, y, s):
    sentence = []
    labels = [0]
    spans = []
    for i in range(len(x)):
      word = x[i]
      label = y[i]
      tokenized_word = self.tokenizer.tokenize(word)
      sentence.extend(tokenized_word)
      labels.extend([label for k in range(len(tokenized_word))])
      curr = s[i][0]
      spans.append([curr, curr+len(tokenized_word[0])])
      curr += len(tokenized_word[0])
      for j in range(len(tokenized_word)-1):
        spans.append([curr, curr+len(tokenized_word[j+1])-2])
        curr += len(tokenized_word[j+1])-2
      spans[-1][-1] = s[i][1]
    labels.append(0)
    return(sentence, labels, spans)

  def get_attention_mask(self, x):
    return([[(i!=0) for i in text] for text in x])

  def prepare_data(self):
    for i in range(len(self.X)):
      self.X[i], self.y[i], self.spans[i] = self.tokenize_data(self.X[i], self.y[i], self.spans[i])
    self.X = pad_sequences([tokenizer.encode(text) for text in self.X], maxlen = self.maxlen, dtype='long', value=0.0, truncating='post', padding = 'post')
    self.y = pad_sequences(self.y, maxlen=self.maxlen, value=2, dtype='long', truncating='post', padding='post')
    self.attention_mask = self.get_attention_mask(self.X)
  
  def get_text_lengths(self, masks):
    lengths = []
    for mask in masks:
      lengths.append(torch.sum(mask).item())
    return(lengths)

  def evaluate(self, flag):
    predictions , true_labels = [], []
    for batch in self.dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_id, b_input_mask, b_labels = batch
      self.model.eval()
      
      if(flag==1):
        with torch.no_grad():
          outputs = self.model(b_input_id, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs[1].detach().cpu().numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      
      elif(flag==2):
        with torch.no_grad():
          out = model(b_input_id, b_input_mask)
          logits = torch.argmax(F.softmax(out, dim=2), dim=2).to('cpu').numpy()
          predictions.extend(logits)
      
      else:
        preds = model(b_input_id, b_input_mask, None, True)
        predictions.extend([p for p in preds])

      true_labels.extend(b_labels)
    if(flag!=3):
      self.pred_tags = [[p_i for p_i, l_i in zip(p, l) if l_i != 2][1:-1] for p, l in zip(predictions, true_labels)]
    else:
      self.pred_tags = predictions

  def f1score(self):
    self.f1list = []
    self.predicted_spans = []
    f1 = 0
    for i in range(len(self.spans)):
      s = [0 for k in range(len(val_data['text'][i]))]
      prev = 0
      for j in range(len(self.spans[i])):
        for k in range(self.spans[i][j][0], self.spans[i][j][1]):
          s[k] = self.pred_tags[i][j]
        if(prev==1 and self.pred_tags[i][j]==1):
          for l in range(self.spans[i][j-1][1], self.spans[i][j][0]):
            s[l] = 1
        
        prev = self.pred_tags[i][j]
      self.f1list.append(f1_score(self.target_spans[i], s))
      f1 += f1_score(self.target_spans[i], s, zero_division=1)
      self.predicted_spans.append(s)
    return(f1/len(self.X))

In [ ]:
train_path = 'drive/MyDrive/Ensemble/train.pkl'
val_path = 'drive/MyDrive/Ensemble/val.pkl'

with open(train_path, 'rb') as f:
  train_data = pickle.load(f)
  f.close()

with open(val_path, 'rb') as f:
  val_data = pickle.load(f)
  f.close()
X = val_data['token_final'].values
Y = val_data['target_final'].values
spans = val_data['span_final'].values
target_spans = []
for i in range(len(X)):
  s = [0 for j in range(len(val_data['text'][i]))]
  for k in val_data['spans'][i]:
    s[k] = 1
  target_spans.append(s)

In [ ]:
model = torch.load('drive/MyDrive/Toxic Spam/1612augroberta2.pt', map_location=torch.device('cpu'))


In [ ]:
metric = SpanAvgF1(model, tokenizer, X, Y, spans, target_spans, 500)
metric.evaluate(1)
metric.f1score()

0.6243990363167001

In [ ]:
# with open(train_path, 'rb') as f:
#   train_data = pickle.load(f)
#   f.close()

# with open(val_path, 'rb') as f:
#   val_data = pickle.load(f)
#   f.close()

# X = val_data['token_final'].values
# Y = val_data['target_final'].values
# spans = val_data['span_final'].values
# target_spans = []
# for i in range(len(X)):
#   s = [0 for j in range(spans[i][-1][-1])]
#   for k in val_data['spans'][i]:
#     s[k] = 1
#   target_spans.append(s)

In [ ]:
# metric = SpanAvgF1(model, tokenizer, X, Y, spans, target_spans)
# metric.f1score()